<a href="https://colab.research.google.com/github/ab-rohman/Computer_Vision/blob/main/BigDataQUIZ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 45.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bc00cda85adfcd4e646b6ad6d0e4d8667af32f1d2d27fea7baafe4793f3bf136
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

from pyspark.ml  import Pipeline     
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
  .master("local")\
  .appName("Colab")\
  .config('spark.ui.port', '4050')\
  .getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/BigData/CAvideos.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)



In [ ]:
df.show(4,False)

+-----------+-------------+-----------------------------------------------------+-------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#define data schema for file we want to read
youtubeSchema = StructType([
    StructField("video_id", StringType(), True),
    StructField("trending_date", StringType(), True),
    StructField("title", StringType(), True),
    StructField("channel_title", StringType(), True),
    StructField("category_id", StringType(), True),
    StructField("publish_time", StringType(), True),
    StructField("tags", StringType(), True),
    StructField("views", IntegerType(), True),
    StructField("likes", IntegerType(), True),
    StructField("dislikes", IntegerType(), True),
    StructField("comment_count", IntegerType(), True),
    StructField("thumbnail_link", StringType(), True),
    StructField("comments_disabled", BooleanType(), True),
    StructField("ratings_disabled", BooleanType(), True),
    StructField("video_error_or_removed", BooleanType(), True),
    StructField("description", StringType(), True),
])    

In [ ]:
df2 = spark.read.option("header",True).csv("/content/drive/MyDrive/BigData/CAvideos.csv",schema=youtubeSchema)

In [ ]:
df2.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- video_error_or_removed: boolean (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
df2.show()

+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|                 title|       channel_title|category_id|        publish_time|                          tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|n1WpP7iowLc|     17.14.11|  Eminem - Walk On ...|          EminemVEVO|         10|2017-11-10T17:00:...|          "Eminem"|"Walk"|"...|17158579| 787425|   43420|       1258

In [ ]:
df2.select("video_id", "trending_date", "title").show(5)

+-----------+-------------+--------------------+
|   video_id|trending_date|               title|
+-----------+-------------+--------------------+
|n1WpP7iowLc|     17.14.11|Eminem - Walk On ...|
|0dBIkQ4Mz1M|     17.14.11|PLUSH - Bad Unbox...|
|5qpjK5DgCt4|     17.14.11|Racist Superman |...|
|d380meD0W0M|     17.14.11|I Dare You: GOING...|
|2Vv-BfVoq4g|     17.14.11|Ed Sheeran - Perf...|
+-----------+-------------+--------------------+
only showing top 5 rows



In [ ]:
df2.select('channel_title').distinct().count()

5154

In [ ]:
df2.orderBy(['views'], ascending = [False]).show()

+-----------+-------------+-------------------------+--------------------+-----------+--------------------+------------------------+---------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+-------------------------+
|   video_id|trending_date|                    title|       channel_title|category_id|        publish_time|                    tags|    views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|              description|
+-----------+-------------+-------------------------+--------------------+-----------+--------------------+------------------------+---------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+-------------------------+
|FlsCjmMhFmw|     17.13.12|     YouTube Rewind: T...|   YouTube Spotlight|         24|2017-12-06T17:58:...|    "Rewind"|"Rewind ...|137843120|3014479| 1602383|    

In [ ]:
df2_sorted = df2.orderBy(['views'], ascending = [False])
df2_sorted.select("title","channel_title", "views").show(20, False)

+--------------------------------------------------------------------------+--------------------+---------+
|title                                                                     |channel_title       |views    |
+--------------------------------------------------------------------------+--------------------+---------+
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind                        |YouTube Spotlight   |137843120|
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind                        |YouTube Spotlight   |125431369|
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind                        |YouTube Spotlight   |113876217|
|YouTube Rewind: The Shape of 2017 | #YouTubeRewind                        |YouTube Spotlight   |100911567|
|Childish Gambino - This Is America (Official Video)                       |ChildishGambinoVEVO |98938809 |
|Marvel Studios' Avengers: Infinity War Official Trailer                   |Marvel Entertainment|89930713 |
|Marvel Studios' Avengers: I

In [ ]:
df2_filtered = df2.filter("views > 50000000 AND title IS NOT NULL")
df2_filtered.select("title","channel_title", "views").show(10, False)

+-------------------------------------------------------+--------------------+--------+
|title                                                  |channel_title       |views   |
+-------------------------------------------------------+--------------------+--------+
|Luis Fonsi, Demi Lovato - Échame La Culpa              |LuisFonsiVEVO       |56843038|
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment|56367282|
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment|66637636|
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment|74789251|
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment|80360459|
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment|84281319|
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment|87450245|
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment|89930713|
|YouTube Rewind: The Shape of 20

In [ ]:
df2_sorted2 = df2.orderBy(['dislikes'], ascending = [False])
df2_sorted2.select("channel_title", "dislikes","title").show(20, False)

+-----------------+--------+------------------------------------------------------------------------------------------+
|channel_title    |dislikes|title                                                                                     |
+-----------------+--------+------------------------------------------------------------------------------------------+
|YouTube Spotlight|1602383 |YouTube Rewind: The Shape of 2017 | #YouTubeRewind                                        |
|YouTube Spotlight|1545018 |YouTube Rewind: The Shape of 2017 | #YouTubeRewind                                        |
|YouTube Spotlight|1470387 |YouTube Rewind: The Shape of 2017 | #YouTubeRewind                                        |
|YouTube Spotlight|1353655 |YouTube Rewind: The Shape of 2017 | #YouTubeRewind                                        |
|YouTube Spotlight|1127811 |YouTube Rewind: The Shape of 2017 | #YouTubeRewind                                        |
|YouTube Spotlight|884963  |YouTube Rewi

In [ ]:
df2_filtered2 = df2.filter("dislikes > 200000 AND title IS NOT NULL")
df2_filtered2.select("title","channel_title", "dislikes").show(10, False)

+------------------------------------------------------------------------------------------+-----------------+--------+
|title                                                                                     |channel_title    |dislikes|
+------------------------------------------------------------------------------------------+-----------------+--------+
|Jake Paul - Saturday Night (Song) feat. Nick Crompton & Chad Tepper (Official Music Video)|Jake Paul        |200391  |
|Jake Paul - It's Everyday Bro (Remix) [feat. Gucci Mane]                                  |Jake Paul        |244229  |
|Jake Paul - It's Everyday Bro (Remix) [feat. Gucci Mane]                                  |Jake Paul        |350118  |
|Jake Paul - It's Everyday Bro (Remix) [feat. Gucci Mane]                                  |Jake Paul        |439353  |
|Jake Paul - It's Everyday Bro (Remix) [feat. Gucci Mane]                                  |Jake Paul        |480359  |
|Jake Paul - It's Everyday Bro (Remix) [

In [ ]:
df2_sorted3 = df2.orderBy(['comment_count'], ascending = [False])
df2_sorted3.select("title", "comment_count","tags").show(20, False)

+-------------------------------------------------------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                        |comment_count|tags                                                                                                                                                                                                                                                                                                                                         